In [1]:
import yfinance as yf
import pandas as pd
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta
#['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
#Scrape recent stock price data
def get_stock_data(ticker, period='5d'):
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)
    # Check if the DataFrame is empty and handle the error
    if df.empty:
        raise ValueError("No data found for the given ticker and period.")
    return df

#Apply linear regression
def apply_linear_regression(df):
    df.reset_index(inplace=True)  # Reset index to create a 'Date' column
    df['OrdinalDate'] = df['Date'].map(datetime.toordinal)  # Add a new column for ordinal dates

    X = df['OrdinalDate'].values.reshape(-1, 1)
    y = df['Close'].values

    model = LinearRegression()
    model.fit(X, y)

    next_day = (datetime.now() + timedelta(days=1)).toordinal()
    prediction = model.predict([[next_day]])

    return prediction[0]

#Store data and prediction in CSV
def save_to_csv(df, prediction, filename='stock_data.csv'):
    # Remove the 'OrdinalDate' column if it exists
    if 'OrdinalDate' in df.columns:
        df.drop(columns=['OrdinalDate'], inplace=True)

    # Save original data to stock_data.csv
    df.to_csv(filename, index=False)

    # Save prediction to prediction_stock_data.csv
    next_day = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    prediction_df = pd.DataFrame({
        'Date': [next_day],
        'Prediction': [prediction]
    })
    prediction_df.to_csv(f'prediction_{filename}', index=False)


# Main function to scrape, predict, and save
def main(ticker):
    try:
        df = get_stock_data(ticker)
        prediction = apply_linear_regression(df)
        save_to_csv(df, prediction)
        print(f"Prediction for next day: {prediction}")
    except ValueError as e:
        print(f"Error: {e}")

# Example usage:
ticker = 'GOOGL'  # You can change this to any stock ticker
main(ticker)
#META,GOOGL,AAPL,MSFT,TSLA,EURUSD=X,GBPUSD=X,EURCAD=X,BTC-USD,USDT-USD


Prediction for next day: 182.30101470951922
